In [162]:
!curl -s -X DELETE http://localhost:9200/sqe_metrics/

{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [sqe_metrics]","index":"sqe_metrics","resource.id":"sqe_metrics","resource.type":"index_or_alias","index_uuid":"_na_"}],"type":"index_not_found_exception","reason":"no such index [sqe_metrics]","index":"sqe_metrics","resource.id":"sqe_metrics","resource.type":"index_or_alias","index_uuid":"_na_"},"status":404}

In [163]:
# Hack needed due to not high enough disk space locally
# See reference at bottom of Notebook

# !curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_cluster/settings -d '{ "transient": { "cluster.routing.allocation.disk.threshold_enabled": false } }'

{"acknowledged":true,"persistent":{},"transient":{"cluster":{"routing":{"allocation":{"disk":{"threshold_enabled":"false"}}}}}}

In [151]:
import time
import json
import random
import uuid
import pandas as pd

from opensearchpy import OpenSearch

client = OpenSearch("http://localhost:9200", use_ssl=False)

In [152]:
df = pd.read_csv("metrics.csv")

In [153]:
df.head()

,Unnamed: 0,datetime,search_config,query_id,query,metric,value
0,0,2024-06-25,research_1,4428,4t camo sweat shirt,dcg,0.117057
1,0,2024-06-25,research_1,8077,all day office chair,dcg,1.000000
2,0,2024-06-25,research_1,17666,bluray movie collection,dcg,1.000000
3,0,2024-06-25,research_1,26436,citizen limited edition,dcg,0.071773
4,0,2024-06-25,research_1,30400,cruz comunion niño,dcg,1.891241


In [164]:
bulk = []

for ix, metric in df.iterrows():
    metric_dict = {
        "datetime": metric["datetime"],
        "search_config": metric["search_config"],
        "query_id": metric["query_id"],
        "query": metric["query"],
        "metric": metric["metric"],
        "value": metric["value"],
    }
    event_id = str(uuid.uuid4())
    bulk.append(
        {"index": {"_index": "sqe_metrics", "_id": event_id}}
    )
    bulk.append(metric_dict)
    response = client.bulk(bulk)

In [154]:
df.query_id.size

2240

In [165]:
!curl -s http://localhost:9200/sqe_metrics/_count | jq

{
  "count": 2240,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  }
}


In [161]:
!curl -s -XGET http://localhost:9200/sqe_metrics/_search -H 'Content-Type: application/json' -d' \
{ \
  "query": { \
    "match": { \
      "query": "4t camo sweat shirt" \
    } \
  } \
}' | jq


{
  "took": 43,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 224,
      "relation": "eq"
    },
    "max_score": 10.913962,
    "hits": [
      {
        "_index": "sqe_metrics",
        "_id": "87e876f0-b04a-4722-9719-616123510695",
        "_score": 10.913962,
        "_source": {
          "datetime": "2024-07-01",
          "search_config": "research_1",
          "query_id": 4428,
          "query": "4t camo sweat shirt",
          "metric": "dcg",
          "value": 0.1170574755672868
        }
      },
      {
        "_index": "sqe_metrics",
        "_id": "613de313-5dc7-43da-adc4-a059bd4035d6",
        "_score": 10.913962,
        "_source": {
          "datetime": "2024-06-28",
          "search_config": "research_1",
          "query_id": 4428,
          "query": "4t camo sweat shirt",
          "metric": "ndcg",
          "value": 0.0415687115032911
        }
      

In [166]:
# Hacky, disk space checks need to be disabled on my system to allow Dashboards to work
#
# https://stackoverflow.com/a/57675654

!curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_cluster/settings -d '{ "transient": { "cluster.routing.allocation.disk.threshold_enabled": false } }'
!curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}'

{"acknowledged":true,"persistent":{},"transient":{"cluster":{"routing":{"allocation":{"disk":{"threshold_enabled":"false"}}}}}}{"acknowledged":true}